In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers;
!pip install sentencepiece;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
%cd /content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/lib/
!pwd

/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/lib
/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/lib


In [4]:
from tokenization_kobert import KoBertTokenizer

import json
from copy import deepcopy
import random
from tqdm.notebook import tqdm

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
)



In [5]:
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
add_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [6]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

## 토크나이저 테스팅

In [7]:
new_words = ["씡" , "췬씡" , "궿"]
add_tokenizer.add_tokens(new_words)

3

In [8]:
tstr = "씡췬 췬궿 췬장 궿췬씡"

In [9]:
print(src_tokenizer.tokenize(tstr))

['▁', '씡췬', '▁', '췬궿', '▁', '췬', '장', '▁', '궿췬씡']


In [11]:
src_tokenizer.encode(tstr)

[2, 517, 0, 517, 0, 517, 0, 7178, 517, 0, 3]

In [10]:
print(src_tokenizer.decode(src_tokenizer.encode(tstr)))
print(add_tokenizer.decode(add_tokenizer.encode(tstr)))
print((src_tokenizer.encode(tstr)))
print((add_tokenizer.encode(tstr)))

[CLS] [UNK] [UNK] [UNK]장 [UNK][SEP]
[CLS] 씡 [UNK] [UNK] 궿 [UNK]장 궿 췬씡 [SEP]
[2, 517, 0, 517, 0, 517, 0, 7178, 517, 0, 3]
[2, 8002, 517, 0, 517, 0, 8004, 517, 0, 7178, 8004, 8003, 3]


In [ ]:
print(src_tokenizer.decode(src_tokenizer.encode(tstr)))
print(add_tokenizer.decode(add_tokenizer.encode(tstr)))
print((src_tokenizer.encode(tstr)))
print((add_tokenizer.encode(tstr)))

[CLS] 이다원씨는 자연어처리에 관심이 많으신가요? [UNK]윅윅[SEP]
[CLS] 이다원 씨는 자연어처리 에 관심이 많으신가요? 궿 윅윅[SEP]
[2, 517, 7100, 7020, 6787, 3916, 6855, 7418, 6896, 1091, 517, 6158, 7074, 6733, 5330, 6999, 258, 517, 0, 7057, 7057, 3]
[2, 8002, 3090, 8003, 517, 6896, 1091, 517, 6158, 7074, 6733, 5330, 6999, 258, 8004, 517, 7057, 7057, 3]


In [ ]:
print(add_tokenizer.decode([8003]))

자연어처리


## 모델 불러오기

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    'monologg/kobert',
    'skt/kogpt2-base-v2',
    pad_token_id=trg_tokenizer.bos_token_id
)
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id
model

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['transformer.h.7.crossattention.bias', 'transformer.h.3.crossattention.bias', 'transformer.h.5.crossattention.q_attn.weight', 'transformer.h.9.crossattention.c_proj.weight', 'transformer.h.7.ln_cross_attn.weight', 'transformer.h.5.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.10.crossattention.c_attn.weight', 'transformer.h.2.crossattention.c_proj.weight', 'transformer.h.9.crossattention.bias', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.1.ln_cross_attn.weight', 'transformer.h.5.crossattention.masked_bias', 'transformer.h.4.crossattention.c_proj.weight', 'transformer.h.1.crossattention.bias', 'transformer.h.2.crossattention.bias', 'transformer.h.11.crossattention.bias', 'transformer.h.0.crossattention.q_attn.weight', 'transformer.h.4.crossattention.bias', 'transformer.h.7.crossattention.q_attn.wei

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## 토크나이저 단어추가 및 모델 임베딩 변경.

In [ ]:
!pip install konlpy

from konlpy.tag import Okt

okt = Okt()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/real_word_data.json", 'rb') as f:
    dw_dict = json.load(f)
new_words_d = []
new_words = []
cnt_words = []
for cnt in range(4,9):
    new_words = []
    for i,v in tqdm(enumerate(dw_dict["utterance"])):
        if 0 in src_tokenizer.encode(v["dialect_form"]) and len(src_tokenizer.encode(v["dialect_form"])) == cnt and not (v["dialect_form"] in cnt_words):
            morphs = okt.morphs(v["dialect_form"])
            for m in morphs:
                if not (m in new_words_d):
                    new_words.append(m)
                    new_words_d.append(m)
            cnt_words.append(v["dialect_form"])
            print(src_tokenizer.decode(src_tokenizer.encode(v["dialect_form"])), v["dialect_form"], len(src_tokenizer.encode(v["dialect_form"])))

# 추가할 단어를 토큰화합니다

    # 토크나이저에도 새로운 단어를 추가합니다
    src_tokenizer.add_tokens(new_words)


0it [00:01, ?it/s]

[CLS] 그[UNK][SEP] 그믄 4
[CLS] 어[UNK][SEP] 어떵 4
[CLS] 하[UNK][SEP] 하믄 4
[CLS] [UNK][SEP] 봔 4
[CLS] 봐[UNK][SEP] 봐봔 4
[CLS] [UNK][SEP] 겅 4
[CLS] 되[UNK][SEP] 되믄 4
[CLS] 챙기[UNK][SEP] 챙기잰 4
[CLS] 주[UNK][SEP] 주잰 4
[CLS] 하[UNK][SEP] 하잰 4
[CLS] 게[UNK][SEP] 게믄 4
[CLS] [UNK][SEP] 맻 4
[CLS] 보[UNK][SEP] 보믄 4
[CLS] [UNK][SEP] 겜 4
[CLS] 지지[UNK][SEP] 지지잰 4
[CLS] 해[UNK][SEP] 해봔 4
[CLS] 나[UNK][SEP] 나눵 4
[CLS] 뽑[UNK][SEP] 뽑잰 4
[CLS] 세[UNK][SEP] 세웡 4
[CLS] 몰아[UNK][SEP] 몰아졍 4
[CLS] 해[UNK][SEP] 해놘 4
[CLS] [UNK][SEP] 겡 4
[CLS] 허[UNK][SEP] 허믄 4
[CLS] 짜[UNK][SEP] 짜믄 4
[CLS] 안[UNK][SEP] 안줭 4
[CLS] [UNK][SEP] 놩 4
[CLS] [UNK][SEP] 뒁 4
[CLS] 구[UNK][SEP] 구웡 4
[CLS] 꺼[UNK][SEP] 꺼졍 4
[CLS] 나가[UNK][SEP] 나가믄 4
[CLS] 또[UNK][SEP] 또똣 4
[CLS] 더[UNK][SEP] 더겅 4
[CLS] 아니[UNK][SEP] 아니꽈 4
[CLS] [UNK][SEP] 폿 4
[CLS] 오[UNK][SEP] 오믄 4
[CLS] [UNK][SEP] 됀 4
[CLS] 마[UNK][SEP] 마씸 4
[CLS] 먹[UNK][SEP] 먹잰 4
[CLS] 말이[UNK][SEP] 말이꽈 4
[CLS] 쓰[UNK][SEP] 쓰잰 4
[CLS] 뭐[UNK][SEP] 뭐꽈 4
[CLS] 남[UNK][SEP] 남쪄 4
[CLS] 그[UNK][SEP] 그쟈 4
[CLS] [UNK][SEP] 

0it [00:00, ?it/s]

[CLS] 마[UNK]지[SEP] 마눙지 5
[CLS] [UNK]이[SEP] 밲이 5
[CLS] 골[UNK]이[SEP] 골괭이 5
[CLS] 덜었[UNK][SEP] 덜었졘 5
[CLS] 않[UNK]댄[SEP] 않앴댄 5
[CLS] [UNK]주[SEP] 핬주 5
[CLS] [UNK]써[SEP] 헙써 5
[CLS] 냈[UNK][SEP] 냈졘 5
[CLS] [UNK]니까[SEP] 헙니까 5
[CLS] 살암[UNK][SEP] 살암졘 5
[CLS] [UNK]언[SEP] 훑언 5
[CLS] 괜[UNK][SEP] 괜츈 5
[CLS] 기[UNK]써[SEP] 기댑써 5
[CLS] [UNK]네[SEP] 쫍네 5
[CLS] 쓰[UNK]다[SEP] 쓰겄다 5
[CLS] [UNK]댄[SEP] 땄댄 5
[CLS] [UNK]어야[SEP] 덖어야 5
[CLS] [UNK]으면[SEP] 덖으면 5
[CLS] 시[UNK]는데[SEP] 시껐는데 5
[CLS] [UNK]은[SEP] 덖은 5
[CLS] [UNK]네[SEP] 깼네 5
[CLS] [UNK]서[SEP] 헙서 5
[CLS] [UNK]니다[SEP] 헙니다 5
[CLS] 언치[UNK][SEP] 언치냑 5
[CLS] 치[UNK]샤[SEP] 치웜샤 5
[CLS] [UNK]댄[SEP] 헜댄 5
[CLS] 영[UNK]니까[SEP] 영헙니까 5
[CLS] [UNK]엉[SEP] 싞엉 5
[CLS] [UNK]지도[SEP] 팹지도 5
[CLS] 시[UNK]댄[SEP] 시켬댄 5
[CLS] [UNK]으면[SEP] 쏢으면 5
[CLS] [UNK]낭[SEP] 옻낭 5
[CLS] 개[UNK]낭[SEP] 개옻낭 5
[CLS] 시[UNK]서[SEP] 시켬서 5
[CLS] [UNK]는[SEP] 볿는 5
[CLS] [UNK] 아졍 [SEP] 톧아졍 5
[CLS] 소[UNK]이[SEP] 소늉이 5
[CLS] [UNK]들은[SEP] 둭들은 5
[CLS] [UNK]어[SEP] 줫어 5
[CLS] [UNK]앙[SEP] 쌂앙 5
[CLS] [UNK]덴[SEP] 줫덴 5
[CLS]

0it [00:00, ?it/s]

[CLS] 시커[UNK]게[SEP] 시커맣게 6
[CLS] 냅[UNK]하게[SEP] 냅쌱하게 6
[CLS] [UNK]서예[SEP] 쏩서예 6
[CLS] [UNK]니다게[SEP] 멉니다게 6
[CLS] 그러 니[UNK][SEP] 그러니꼐 6
[CLS] [UNK]니다게[SEP] 큽니다게 6
[CLS] 은[UNK]게[SEP] 은혠게 6
[CLS] 안[UNK]주게[SEP] 안줫주게 6
[CLS] [UNK]째번[SEP] 츳째번 6
[CLS] 배와[UNK]고[SEP] 배와젔고 6
[CLS] [UNK]었쥬[SEP] 걲었쥬 6
[CLS] 기[UNK]이로[SEP] 기곙이로 6
[CLS] 예[UNK]신가[SEP] 예뻠신가 6
[CLS] 아[UNK]니다게[SEP] 아픕니다게 6
[CLS] [UNK]어젼[SEP] 짊어젼 6
[CLS] [UNK]덜게[SEP] 둭덜게 6
[CLS] [UNK]덜이[SEP] 둭덜이 6
[CLS] [UNK]주게[SEP] 냇주게 6
[CLS] 생[UNK]수다[SEP] 생겻수다 6
[CLS] [UNK]수과[SEP] 줫수과 6
[CLS] [UNK]저게[SEP] 뒛저게 6
[CLS] [UNK]신게[SEP] 뒘신게 6
[CLS] 사줘[UNK]서[SEP] 사줘븝서 6
[CLS] [UNK]다네[SEP] 켯다네 6
[CLS] 아까[UNK]직[SEP] 아까웜직 6
[CLS] 놔[UNK]당[SEP] 놔둿당 6
[CLS] 하[UNK]디다[SEP] 하큽디다 6
[CLS] [UNK]수다[SEP] 텄수다 6
[CLS] 되있당[UNK][SEP] 되있당꼐 6
[CLS] [UNK]으고[SEP] 얕으고 6
[CLS] [UNK]치면[SEP] 벋치면 6
[CLS] 놉 [UNK]덴[SEP] 놉닸덴 6
[CLS] 며[UNK]날[SEP] 며칟날 6
[CLS] 와[UNK]주[SEP] 와벴주 6
[CLS] 엣[UNK]디[SEP] 엣뒨디 6
[CLS] [UNK]서게[SEP] 툽서게 6
[CLS] [UNK]굴엉[SEP] 뒹굴엉 6
[CLS] [UNK]으 믄 [SEP] 짦으믄 6
[CLS] [UNK]주게[

0it [00:00, ?it/s]

[CLS] [UNK]게만은[SEP] 씅게만은 7
[CLS] 안[UNK]던거라[SEP] 안줫던거라 7
[CLS] [UNK]앙으네[SEP] 곪앙으네 7
[CLS] [UNK]단주난[SEP] 곴단주난 7
[CLS] [UNK]랑보라[SEP] 뷀랑보라 7
[CLS] [UNK]불이랜[SEP] 굼불이랜 7
[CLS] [UNK]어져 신디 [SEP] 걲어져신디 7
[CLS] 동그[UNK]허게[SEP] 동그랗허게 7
[CLS] [UNK]기는거라[SEP] 맫기는거라 7
[CLS] 놔[UNK]저게[SEP] 놔둿저게 7
[CLS] [UNK] 가락 이연[SEP] 숟가락이연 7
[CLS] 잊어[UNK]져[SEP] 잊어벴져 7
[CLS] [UNK]망이네[SEP] 썲망이네 7
[CLS] 뚜[UNK]하다게[SEP] 뚜뚯하다게 7
[CLS] 나[UNK]주마시[SEP] 나뻣주마시 7
[CLS] 새까[UNK]해여[SEP] 새까맣해여 7
[CLS] 새까[UNK]한데[SEP] 새까맣한데 7
[CLS] [UNK]어내멍[SEP] 읊어내멍 7
[CLS] [UNK]어대난[SEP] 읊어대난 7
[CLS] 놔[UNK]서라[SEP] 놔덨서라 7
[CLS] 산[UNK]부리처럼[SEP] 산굼부리처럼 7
[CLS] 와사[UNK] 디다 [SEP] 와사쿱디다 7
[CLS] [UNK]앙으네[SEP] 욖앙으네 7
[CLS] [UNK]니께게[SEP] 뭡니께게 7
[CLS] [UNK]탱이하고[SEP] 귓탱이하고 7
[CLS] [UNK]고락씩[SEP] 숟고락씩 7
[CLS] [UNK]작이라는디[SEP] 띵작이라는디 7
[CLS] [UNK]데레라[SEP] 츤데레라 7
[CLS] [UNK]라지게[SEP] 뺼라지게 7
[CLS] [UNK]비기낭[SEP] 쑴비기낭 7
[CLS] [UNK]을적에[SEP] 썲을적에 7
[CLS] [UNK]니데이[SEP] 됍니데이 7
[CLS] [UNK]거렸주[SEP] 끙끙거렸주 7
[CLS] 바[UNK]수다게[SEP] 바꽘수다게 7
[CLS] 오지[UNK]파연[SEP] 오지랖파연 7
[CLS] 괜[UNK]아젼

0it [00:00, ?it/s]

[CLS] 섭[UNK]한거라[SEP] 섭췰한거라 8
[CLS] 화 해시 [UNK]수게[SEP] 화해시켯수게 8
[CLS] 콩[UNK]거추룩[SEP] 콩닢거추룩 8
[CLS] [UNK]아져불고[SEP] 똟아져불고 8
[CLS] [UNK]거우다게[SEP] 쎌거우다게 8
[CLS] 솔[UNK]카부댄[SEP] 솔찟카부댄 8
[CLS] 뽀슬[UNK]수다[SEP] 뽀슬뢌수다 8
[CLS] 고라 [UNK]대난게[SEP] 고라짔대난게 8
[CLS] 먹어짐[UNK]커다[SEP] 먹어짐짔커다 8
[CLS] [UNK]화했거든[SEP] 젆화했거든 8
[CLS] [UNK]어와불든지[SEP] 뻇어와불든지 8
[CLS] [UNK]뒀당은에[SEP] 놧뒀당은에 8
[CLS] [UNK]어와 나서 덜[SEP] 끋어와나서덜 8
[CLS] 다 솖 아[UNK]거든[SEP] 다솖아빘거든 8
[CLS] [UNK]슈카라 고이 [SEP] 샥슈카라고이 8
[CLS] 다[UNK]으 넹 에[SEP] 다퉜으넹에 8
[CLS] [UNK]피어났져[SEP] 궘피어났져 8
[CLS] [UNK]댄마씀게[SEP] 꼇댄마씀게 8
[CLS] [UNK]아본거라[SEP] 삮아본거라 8
[CLS] [UNK]터져나사[SEP] 냇터져나사 8
[CLS] 매[UNK]다 그넹 에[SEP] 매꼇다그넹에 8
[CLS] [UNK]여졌주기게[SEP] 셖여졌주기게 8
[CLS] [UNK]겨지민양[SEP] 욍겨지민양 8
[CLS] 가[UNK] 주마 는[SEP] 가냤주마는 8
[CLS] [UNK]허덴마씨[SEP] 궙궙허덴마씨 8
[CLS] [UNK]게해가지고[SEP] 묽게해가지고 8
[CLS] [UNK]카이도 마씸 [SEP] 홋카이도마씸 8


In [ ]:
# # 토크나이저에도 새로운 단어를 추가합니다
# src_tokenizer.add_tokens(new_words)

# 인코더의 임베딩 크기를 조정합니다
model.encoder.resize_token_embeddings(len(src_tokenizer))

# 추가한 단어가 토크나이저에 적용되었는지 확인합니다
print(src_tokenizer.decode(src_tokenizer.encode(new_words[0])))

[CLS] 섭췰한거 [SEP]


In [ ]:
# # 토크나이저들을 모델에 저장합니다
model.config.encoder_attention_type = 'full'
model.config.decoder_attention_type = 'full'
model.config.src_vocab_size = len(src_tokenizer)
model.config.tgt_vocab_size = len(trg_tokenizer)

# push_to_hub() 함수를 사용하여 모델을 Hugging Face Hub에 업로드합니다
model.push_to_hub("ossp-v0_2")


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

TypeError: ignored

In [ ]:
import pickle
with open('save_edm.pickle', 'wb') as f:
    pickle.dump(model, f)
with open('save_src_tk.pickle', 'wb') as f:
    pickle.dump(src_tokenizer, f)
with open('save_trg_tk.pickle', 'wb') as f:
    pickle.dump(trg_tokenizer, f)

In [ ]:
with open('no_commit/save_src_tk.pickle','rb') as f:
    add_tokenizer=pickle.load(f)
    

In [ ]:
%cd ..

/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai


In [ ]:
print(len(add_tokenizer))

12291


## 형태소 분석기 테스트

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#install mecab
from konlpy.tag import Kkma, Komoran, Okt, Mecab

#mec = Mecab()
okt = Okt()
kkm = Kkma()
kom = Komoran()

In [ ]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/real_word_data.json", 'rb') as f:
    dw_dict = json.load(f)
cnt_words = []

import time
for i,v in tqdm(enumerate(dw_dict["utterance"])):
    if 0 in src_tokenizer.encode(v["dialect_form"]) and not (v["dialect_form"] in cnt_words):
        cnt_words.append(v["dialect_form"])
        print(v["dialect_form"])
        print(okt.morphs(v["dialect_form"]))
        print("#"*50)

0it [00:00, ?it/s]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
차가와수꽈
['차가와', '수', '꽈']
['▁차', '가', '와', '수', '꽈']
##################################################
무사마씸게
['무사마씸', '게']
['▁무', '사', '마', '씸', '게']
##################################################
겅해부나네
['겅', '해부', '나네']
['▁', '겅', '해', '부', '나', '네']
##################################################
이추룩하믄
['이', '추', '룩', '하믄']
['▁이', '추', '룩', '하', '믄']
##################################################
쭐면
['쭐', '면']
['▁', '쭐', '면']
##################################################
쭐었어이
['쭐었', '어', '이']
['▁', '쭐', '었', '어', '이']
##################################################
냄쪄
['냄', '쪄']
['▁', '냄', '쪄']
##################################################
없수꽈
['없수꽈']
['▁없', '수', '꽈']
##################################################
돌앙댕겸주
['돌', '앙', '댕겸주']
['▁돌', '앙', '댕', '겸', '주']
##################################################
물엉댕기잖아
['물', '엉', '댕기', '잖아']
['▁물', '엉', '댕', '기', '잖', '아']
###############################################

KeyboardInterrupt: ignored

In [ ]:
print(len(src_tokenizer))
print(len(add_tokenizer))

12291
8002
